In [1]:
import pandas as pd
import xlwings as xw
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from itertools import chain
from tqdm import tqdm

In [2]:
url = 'http://www.rlsnet.ru/drugs/ukazatel/c0'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)

In [4]:
print(response.status_code)

200


In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
links_div = soup.find('div', {'id': 'letters-ru'})

In [7]:
links = links_div.find_all('a')

In [8]:

full_list = []
for link in links:
    url = link.get('href')
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    print(response.status_code, url)
    main_div = soup.find('div', {'class': 'tab-content'})
    drugs_div = main_div.find_all('div', class_='b-alphabet-list')
    drugs_list = []
    data2 = '' 
    for data in drugs_div:
        data1 = data.find_all('a', class_='link')        
        for i in range(len(data1)):
            data2 = data1[i].text.strip().split('\n')
            for k in data2:
                name = k.strip().split('\n')
                drugs_list.append(name)
    drugs_list = list(chain.from_iterable(drugs_list))
    full_list.append(drugs_list)
    time.sleep(3)

200 https://www.rlsnet.ru/drugs/ukazatel/c0
200 https://www.rlsnet.ru/drugs/ukazatel/c1
200 https://www.rlsnet.ru/drugs/ukazatel/c2
200 https://www.rlsnet.ru/drugs/ukazatel/c3
200 https://www.rlsnet.ru/drugs/ukazatel/c4
200 https://www.rlsnet.ru/drugs/ukazatel/c5
200 https://www.rlsnet.ru/drugs/ukazatel/c6
200 https://www.rlsnet.ru/drugs/ukazatel/c7
200 https://www.rlsnet.ru/drugs/ukazatel/c8
200 https://www.rlsnet.ru/drugs/ukazatel/c9
200 https://www.rlsnet.ru/drugs/ukazatel/ca
200 https://www.rlsnet.ru/drugs/ukazatel/cb
200 https://www.rlsnet.ru/drugs/ukazatel/cc
200 https://www.rlsnet.ru/drugs/ukazatel/cd
200 https://www.rlsnet.ru/drugs/ukazatel/ce
200 https://www.rlsnet.ru/drugs/ukazatel/cf
200 https://www.rlsnet.ru/drugs/ukazatel/d0
200 https://www.rlsnet.ru/drugs/ukazatel/d1
200 https://www.rlsnet.ru/drugs/ukazatel/d2
200 https://www.rlsnet.ru/drugs/ukazatel/d3
200 https://www.rlsnet.ru/drugs/ukazatel/d4
200 https://www.rlsnet.ru/drugs/ukazatel/d5
200 https://www.rlsnet.ru/drugs/

In [9]:
final_list = list(chain.from_iterable(full_list))

In [10]:
len(final_list)

26902

In [11]:
df = pd.DataFrame(final_list)

In [12]:
df.rename(columns = {0:'Название л/с'}, inplace = True)

In [13]:
df['Название л/с'] =df['Название л/с'].str.replace('®', '')

In [14]:
df = df.loc[~(df['Название л/с'].str[0].isin(['(']))&~(df['Название л/с'].str[-1].isin([')']))]

In [15]:
df = df.reset_index(drop=True)

In [16]:
df.to_excel('Список лекарств.xlsx')

Выгрузка групп для ЛС

In [23]:
df1 = pd.read_excel('Список лекарст.xlsx')

In [24]:
drugs_name_list = df1['Название л/с'].tolist()

In [25]:
len(drugs_name_list)

15343

In [26]:
driver = webdriver.Chrome() 
driver.get("https://www.rlsnet.ru")

In [27]:
drugs_gpoup_dict = {}
for i in tqdm(range(0, 10)): #Выбор среза
    element = driver.find_element(By.CLASS_NAME, 'form-control')
    element.send_keys(drugs_name_list[i], Keys.ENTER)
    time.sleep(3)
    group_list = []
    elem_list = []
    try:
        div_1 = driver.find_elements(By.CLASS_NAME, 'block')
        for tag in div_1:
            tag_name = tag.text
            gr = 'В фармакологических группах'
            el = 'В действующих веществах'
            if gr in tag_name:
                f_group = tag.find_element(By.TAG_NAME, 'ul')
                farm_group = f_group.text
                group_list.append(farm_group)
            else:
                pass
            if el in tag_name:
                el_group = tag.find_element(By.TAG_NAME, 'ul')
                elem_group = el_group.text
                elem_list.append(elem_group)
            else:
                pass
    except:
        print('Не найдено')
    group_str = (''.join(group_list))
    group_str = group_str.replace('\n', '; ')
    elem_str = (''.join(elem_list))
    elem_str = elem_str.replace('\n', '; ')
    drugs_gpoup_dict.update({drugs_name_list[i]: [group_str, elem_str]})
temp_df1 = pd.DataFrame(drugs_gpoup_dict)
temp_df1 = temp_df1.T
temp_df1.rename(columns = {0:'Фарм_группа', 1:'Действующее_вещество'}, inplace = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:34<00:00,  3.47s/it]


In [28]:
len(temp_df1)

10

In [29]:
temp_df1

,Фарм_группа,Действующее_вещество
А.Т.10,Корректоры метаболизма костной и хрящевой ткан...,Дигидротахистерол
Абаджио,Иммунодепрессанты,Терифлуномид
Абакавир,Средства для лечения ВИЧ-инфекции; Противовиру...,Абакавир; Абакавир + Зидовудин + Ламивудин; Аб...
Абакавир + Зидовудин + Ламивудин,Средства для лечения ВИЧ-инфекции в комбинациях,Абакавир + Зидовудин + Ламивудин; Абакавир; Зи...
Абакавир Канон,Средства для лечения ВИЧ-инфекции,Абакавир
Абакавир-АВС,Средства для лечения ВИЧ-инфекции,Абакавир
Абакавир-Эдвансд,Средства для лечения ВИЧ-инфекции,Абакавир
Абакавир/Ламивудин-Тева,Средства для лечения ВИЧ-инфекции в комбинация...,Абакавир + Ламивудин; Абакавир; Ламивудин
Абакавир+Ламивудин,,Абакавир + Ламивудин; Абакавир + Ламивудин + З...
Абакавир+Ламивудин Сан,,


In [ ]:
temp_df1.to_excel('ЛС1.xlsx')